In [ ]:
# FUNCTION: “GET FROM INTERNET” 
# Funktion needs (Stock, YearGDP)
# scrape from the net:
# GDP growth (real)
# Expected Inflation Rate, https://ec.europa.eu/info/business-economy-euro/economic-performance-and-forecasts/economic-performance-country/euro-area_en
# Beta for the stock
# dividend for the stock
# German 10 Year Bond Rate (-0,25%)
# Return On Equity (In Percent)
#
# check data is usable type, change type if needed.
#
# FUNCTION RETURN: (GDP Growth, ExpInflRate, Beta, Dividend, GerTenYRate, ROE)